# Draw Annotation Boxes

In [28]:
annot_path = "/home/randy/Documents/HuaweiICT/repo/ICT-Huawei/action_video/my_dataset/ann/data1_12.mp4.json"
video_path = "/home/randy/Documents/HuaweiICT/repo/ICT-Huawei/action_video/my_dataset/video/data1_12.mp4"
fps = 12

In [63]:
class_action_map = {
    "person_1" : [],
    "person_2" : [],
    "person_3" : [],
    "person_4" : [],
    "person_5" : [],
    "person_6" : [],
    "person_7" : [],
    "person_8" : [],
}

for i in range(126):
    # Person 1
    action_person_1 = ["walking"]
    if i >= 7:
        action_person_1.append("staring")
    class_action_map["person_1"].append(action_person_1)

    # Person 2
    action_person_2 = []
    if i <= 45 or i >= 55:
        action_person_2.extend(["standing","staring"])
    class_action_map["person_2"].append(action_person_2)
    
    # Person 3
    action_person_3 = ["crouch"]
    class_action_map["person_3"].append(action_person_3)

    # Person 4
    action_person_4 = []
    if i <= 16:
        action_person_4.extend(["standing","staring","talking"])
    if i <= 25:
        action_person_4.extend(["walking"])
    if i >= 29 and i <= 37:
        action_person_4.extend(["grabbing","positioning"])
    if i >= 30:
        action_person_4.extend(["pushing"])
    class_action_map["person_4"].append(action_person_4)
    
    # Person 5
    action_person_5 = []
    if i <= 26:
        action_person_5.extend(["standing","talking"])
    if i >= 29 and i <= 37:
        action_person_5.extend(["grabbing","positioning"])
    if i >= 30:
        action_person_5.extend(["pulling"])
    class_action_map["person_5"].append(action_person_5)

    # Person 6
    action_person_6 = []
    if i <= 20:
        action_person_6.extend(["slight bowing","staring"])
    if i >= 21:
        action_person_6.extend(["crouch","staring"])
    class_action_map["person_6"].append(action_person_6)

    # Person 7
    action_person_7 = ["walking"]
    class_action_map["person_7"].append(action_person_7)

    # Person 8
    action_person_8 = ["walking"]
    class_action_map["person_8"].append(action_person_8)


In [67]:
import cv2
import json
import os
import numpy as np

# Load the video
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Read the annotation file
with open(annot_path, 'r') as f:
    annotations = json.load(f)

# Read the meta file
meta_path = "./meta.json"
with open(meta_path,'r') as f:
    meta = json.load(f)

objects = annotations["objects"]

color_maps = {el["title"] : el["color"] for el in meta["classes"] if el["title"].startswith("person")}

object_action_map = {}
object_cmap = {}
for obj in objects:
    key = obj["key"]
    class_title = obj["classTitle"]
    try:
        hex_color = color_maps[class_title]
        rgb_color = tuple(int(hex_color[i:i+2], 16) for i in (1, 3, 5))
        object_cmap[key] = rgb_color
        object_action_map[key] = class_action_map[class_title]
    except:
        rgb_color = tuple(np.random.randint(0,255) for i in range(3))
        object_cmap[key] = rgb_color

result_video_path = os.path.split(video_path)[-1].replace(".mp4","_annotated.mp4")
video_writer = cv2.VideoWriter(result_video_path,
                            cv2.VideoWriter_fourcc(*'mp4v'), 
                            fps, (width, height))

num_frames_video = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
# Loop over each frame in the video
for n in range(num_frames_video):
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Get the annotations for the current frame
        frame_number = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        frame_annotations = annotations["frames"][frame_number]
    except:
        break

    # Draw bounding boxes around each object in the frame
    for fig in frame_annotations["figures"]:
        [x1, y1], [x2, y2] = fig['geometry']['points']['exterior']
        obj_key = fig["objectKey"]
        cv2.rectangle(frame, (x1, y1), (x2, y2), object_cmap[obj_key], 2)

        object_key = fig["objectKey"]

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.3
        thickness = 1
        action = ','.join(object_action_map[object_key][frame_number])
        text_size = cv2.getTextSize(action, font, font_scale, thickness)[0]
        cv2.putText(frame, action, (x1, y1 - text_size[1]), font, font_scale, (0, 255, 0), thickness)
    
    # Write the frame to the output video
    video_writer.write(frame)

    # Display the annotated frame
    # cv2.imshow('frame', frame)
    if cv2.waitKey(60) & 0xFF == ord('q'):
        break

video_writer.release()
# # Clean up
cap.release()
cv2.destroyAllWindows()

print(f"{result_video_path} is Done!")

data1_12_annotated.mp4 is Done!
